Importamos la librerias

In [ ]:

from numpy import sqrt, mean, absolute, real, conj
import numpy as np
import matplotlib.pyplot as plt

from scipy import fftpack
import math as m
import pandas as pd

Traemos la informacion de github

In [ ]:
!wget  https://github.com/SebastianVillega5/ProyectoIA/blob/346f5331fe3033a573a605d69c06b5b2c169330d/Machine%20learning%20PQ/Data%20Base/Normal1.zip?raw=true
!wget https://github.com/SebastianVillega5/ProyectoIA/blob/main/Machine%20learning%20PQ/Data%20Base/sag2.zip?raw=true

--2023-05-22 22:27:28--  https://github.com/SebastianVillega5/ProyectoIA/blob/346f5331fe3033a573a605d69c06b5b2c169330d/Machine%20learning%20PQ/Data%20Base/Normal1.zip?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/SebastianVillega5/ProyectoIA/raw/346f5331fe3033a573a605d69c06b5b2c169330d/Machine%20learning%20PQ/Data%20Base/Normal1.zip [following]
--2023-05-22 22:27:28--  https://github.com/SebastianVillega5/ProyectoIA/raw/346f5331fe3033a573a605d69c06b5b2c169330d/Machine%20learning%20PQ/Data%20Base/Normal1.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/SebastianVillega5/ProyectoIA/346f5331fe3033a573a605d69c06b5b2c169330d/Machine%20learning%20PQ/Data%20Base/Normal1.zip [following]
--2023-05-22 22:27:28--  https://raw.githubusercontent

Leemos los archivos de las ondas sanas

In [ ]:
! unzip /content/Normal1.zip?raw=true
readerN=pd.read_csv("Normal1.csv")
xN=list(np.transpose(np.array(readerN["0"])))

Archive:  /content/Normal1.zip?raw=true
  inflating: Normal1.csv             


Leemos los archivos de las ondas con sag

In [ ]:
! unzip /content/sag2.zip?raw=true
readers=pd.read_csv("sag2.csv")
x=list(np.transpose(np.array(readers["0"])))

Archive:  /content/sag2.zip?raw=true
  inflating: sag2.csv                


Definimos las primeras funciones y las probamos

In [ ]:
def std(vecstd):
  #Funcion para la desviacion estandar
  return np.std(vecstd)
def mean(meanvec):
  #Funcion para el promedio 
  return np.mean(meanvec)
print("La desviacion estandar y el promedio para onda con sag",std(x),mean(x))
print("La desviacion estandar y el promedio sano es",std(xN),mean(xN))
def counth(vech):
  #Funcion para hacer la cuenta de los valores encima de 1
  vecn1=abs(np.array(vech))
  count1=0
  for i in vecn1:
    if abs(i)>1:
      count1=count1+1
  return count1
def countlow(vecl):
  #Funcion para contar las veces que el valor es menor a 0.1
  veclow=abs(np.array(vecl))
  clow=0
  for i in veclow:
    if abs(i)<=0.1:
      clow=clow+1
  return clow
def countinter(vint):
  #Funcion para contar las veces que se encuentra en el intervalo 0.1 y 1
  vinter=abs(np.array(vint))
  cinter=0
  for i in vinter:
    if abs(i)>0.1 and abs(i)<=1:
          cinter=cinter+1
  return cinter
##Probamos la funcion 
print("Las veces que la onda con sag es mayor a 1, menor a 0.1 y entre 0.1 y 1")
print("Es respectivamente")
print(counth(x),countlow(x),countinter(x))

###Le haremos la FFT para sacar una caracteristica llamada THD


La desviacion estandar y el promedio para onda con sag 0.5901470904089854 0.009662840927910967
La desviacion estandar y el promedio sano es 0.7073255822066349 4.909283461214134e-05
Las veces que la onda con sag es mayor a 1, menor a 0.1 y entre 0.1 y 1
Es respectivamente
0 45 488


Probamos la funcion de medicion de la distorsion de la onda que funciona en conjunto con FFT.

In [ ]:
def RMS(Vrms1):
  Vrms= np.abs(Vrms1)
  return sqrt(sum(np.array(Vrms)**2))
def THD(Vthd):
  Vthd= np.abs(Vthd)
  
  return sqrt(((RMS(Vthd)/list(Vthd)[1])**2)-1)*100


def THDIA(vTHDIA1):
  "Esta funcion me saca el THD promedio de todos los ciclos, el menor y el mayor"
  #Dividimos el vector en su numero de ciclos en una lista 
  vTHDIA=list(vTHDIA1)
  interval=[]
  ffi=[]
  THDr=[]
  LHDr=[]
  for i in range(1,m.floor(len(vTHDIA)/53.3)-1):
      num1=m.floor(53*(i-1))
      num2=m.floor(53*(i))
      
      interval=vTHDIA[num1:num2]
      #Hacemos la transformada de fourier y lo añadimos a una lista
      ffi=fftpack.fft(interval)
      
      #Le aplicamos el THD
      THDr.append(THD(list(ffi)[0:7]))
      #Le aplicamos el LHD que es THD pero considerando solo los primeros 7 frecuencias
  
  #Vamos a retornar una lista de 6 numeros 
  mp=[np.mean(THDr),min(THDr),max(THDr)]
  return mp
print("El THD con sag es en promedio ,minimo y maximo:")
print(THDIA(x))
print("El THD para sana es en promedio,minimo y maximo:")
print(THDIA(xN))

from scipy import fftpack
import math as m
import matplotlib.pyplot as plt


El THD con sag es en promedio ,minimo y maximo:
[6.54787757521943, 1.069849543936547, 34.187524645748674]
El THD para sana es en promedio,minimo y maximo:
[1.0818127758425846, 1.0675007242626917, 1.1018815593218223]


Organizamos los datos en un vector para poder enviarselo a la IA en un futuro.

In [ ]:
def preproceso(dfs):
  x=[]
  for i in range(len(dfs.T)-1):
    ldfs=list(np.transpose(np.array(dfs[str(i)])))
    carac=[] 
    carac.append(std(ldfs))
    carac.append(mean(ldfs))
    carac.append(counth(ldfs))
    carac.append(countlow(ldfs))
    carac.append(countinter(ldfs))
    TH=THDIA(ldfs)
    carac.append(TH[0])
    carac.append(TH[1])
    carac.append(TH[2])
    x.append(carac)
  return x
RN=preproceso(readerN)
print("Los datos extraidos de la primera columna son:")
print(RN[0])
print("El total de vectores normales son:",len(RN))
print("El total de caracteristicas son:",len(RN[0]))

Los datos extraidos de la primera columna son:
[0.7073255822066349, 4.909283461214134e-05, 0, 33, 500, 1.0818127758425846, 1.0675007242626917, 1.1018815593218223]
El total de vectores normales son: 20000
El total de caracteristicas son: 8


Armamos una base de datos de las carcateristicas con las sanas y las dañadas con 0 para dañada y 1 para sana:

In [ ]:
def data(dataS,dataN):
  xS=preproceso(dataS)
  xN=preproceso(dataN)
  xData=[]
  yData=[]
  
  for i in range (len(xS)):
    xData.append(xS[i])
    xData.append(xN[i])
    yData.append(1)
    yData.append(0)
  
  return xData, yData
xtest,ytest=data(readers,readerN)

Miramos la longitud de los vectores entregados y cuantas caracteristicas tiene cada uno

In [ ]:
print("xtest tiene ",len(xtest),"datos")
print("ytest tiene ",len(ytest),"datos")
print("Los primeros dos vectores de xtest son:")
print(xtest[0],xtest[1])
print("Los de ytest:")
print(ytest[0],ytest[1])


xtest tiene  20000 datos
ytest tiene  20000 datos
Los primeros dos vectores de xtest son:
[0.5901470904089854, 0.009662840927910967, 0, 45, 488, 6.54787757521943, 1.069849543936547, 34.187524645748674] [0.7073255822066349, 4.909283461214134e-05, 0, 33, 500, 1.0818127758425846, 1.0675007242626917, 1.1018815593218223]
Los de ytest:
1 0
